<a href="https://colab.research.google.com/github/cric96/DL-exercise/blob/main/GNN_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
%tensorflow_version 1.x
!pip install gnn

import tensorflow as tf
import numpy as np
import gnn.GNN as GNN
import gnn.gnn_utils
import networkx as nx
import scipy as sp
from collections import OrderedDict
import matplotlib.pyplot as plt

TensorFlow 1.x selected.



In [2]:
def weight_variable(shape, nm):
    # function to initialize weights
    initial = tf.truncated_normal(shape, stddev=0.1)
    tf.summary.histogram(nm, initial, collections=['always'])
    return tf.Variable(initial, name=nm)


class Net:
    # class to define state and output network

    def __init__(self, input_dim, state_dim, output_dim):
        # initialize weight and parameter

        self.EPSILON = 0.00000001

        self.input_dim = input_dim
        self.state_dim = state_dim
        self.output_dim = output_dim
        self.state_input = self.input_dim - 1 + state_dim  # removing the id_ dimension

        #### TO BE SET ON A SPECIFIC PROBLEM
        self.state_l1 = 15
        self.state_l2 = self.state_dim

        self.output_l1 = 10
        self.output_l2 = self.output_dim

    def netSt(self, inp):
        with tf.variable_scope('State_net'):
            layer1 = tf.layers.dense(inp, self.state_l1, activation=tf.nn.tanh)
            layer2 = tf.layers.dense(layer1, self.state_l2, activation=tf.nn.tanh)
            return layer2

    def netOut(self, inp):

            layer1 = tf.layers.dense(inp, self.output_l1, activation=tf.nn.relu)
            layer2 = tf.layers.dense(layer1, self.output_l2, activation=tf.nn.relu)

            return layer2

    def Loss(self, output, target, output_weight=None):
        # method to define the loss function
        #lo = tf.losses.softmax_cross_entropy(target, output)
        output = tf.maximum(output, self.EPSILON, name="Avoiding_explosions")  # to avoid explosions
        xent = -tf.reduce_sum(target * tf.log(output), 1)
        lo = tf.reduce_mean(xent)
        return tf.losses.mean_squared_error(target, output)

    def Metric(self, target, output, output_weight=None):
        # method to define the evaluation metric
        correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(target, 1))
        metric = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        return metric

In [3]:
# List of edges in the first graph - last column is the id of the graph to which the arc belongs
def grid(row, column, graph):
  result = []
  for i in range(row - 1):
    for j in range(column - 1):
      index = (row * i) + j
      next_on_column = index + 1
      next_on_row = (row * (i + 1)) + j
      result.append([index, next_on_column, graph])
      result.append([index, next_on_row, graph])
      result.append([index, next_on_row + 1, graph])
  return result

def neighbours(id, edges):
  filtered = filter(lambda x: x[1] == id, edges)
  mapped = map(lambda x: x[0], filtered)
  return list(mapped)

def hop_count(id, edges): 
  visited = set([id])
  od = {}
  hop = 0
  od[id] = hop
  to_visit = set(neighbours(id, edges))
  while (len(to_visit) > 0):
    visited = visited.union(to_visit)
    hop += 1
    next_visit = set([])
    for id_neigh in to_visit:
      od[id_neigh] = hop
      next_visit = next_visit.union(set(neighbours(id_neigh, edges)))
    to_visit = next_visit.difference(visited)
  return dict(sorted(od.items(), key=lambda item: item[0]))

# visualization graph
def plot_graph(E, N, labels):
    g = nx.Graph()
    g.add_nodes_from(range(N.shape[0]))
    g.add_edges_from(E[:, :2])
    node_labels = dict(map(lambda kv: (kv[0], (kv[0], kv[1])), dict(enumerate(labels)).items()))
    nx.draw_spring(g, cmap = plt.get_cmap('Set1'), labels = node_labels)
    plt.show()


# Main file simple

This is the main file for the simple classification task


In [7]:
import tensorflow as tf
import numpy as np
import gnn.GNN as GNN
import gnn.gnn_utils
import networkx as nx
import scipy as sp
from collections import OrderedDict
import matplotlib.pyplot as plt

##### GPU & stuff config
import os

os.environ['CUDA_VISIBLE_DEVICES'] = "0"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

############# data creation ################

# GRAPH #

row = 3
column = 3
#number of nodes
edges = row * column
e = grid(row, column, 0)
# undirected graph, adding other direction
e.extend([[i, j, num] for j, i, num in e])
source_id = 3
result = hop_count(source_id, e).items()
labels = list(map(lambda x : x[1], result))

#reorder
e = sorted(e)
E = np.asarray(e)

# creating node features - simply one-hot values
N = np.zeros((edges, edges), dtype=np.float32)
N[source_id, source_id] = 1
# adding column thta represent the id of the graph to which the node belongs
N = np.concatenate((N, np.zeros((edges,1), dtype=np.float32)),  axis=1)

plot_graph(E,N, labels)
# GRAPH #2

# List of edges in the second graph - last column graph-id
e1 = grid(row, column, 1)
# undirected graph, adding other direction
e1.extend([[i, j, num] for j, i, num in e1])
# reindexing node ids based on the dimension of previous graph (using unique ids)
e2 = [[a + N.shape[0], b + N.shape[0], num] for a, b, num in e1]
#reorder
e2 = sorted(e2)

# Plot second graph
E1 = np.asarray(e1)
N1 = np.zeros((edges, edges), dtype=np.float32)
N1 = np.concatenate((N1, np.zeros((edges, 1), dtype=np.float32)),  axis=1)
source_id = 5
result = hop_count(source_id, e).items()
labels1 = list(map(lambda x : x[1], result))

labels = np.array(labels + labels1)

N1[source_id, source_id] = 1
plot_graph(E1, N1, labels1)

# E = np.concatenate((E, np.asarray(e2)), axis=0)
# N_tot = np.eye(edges + edges_2,  dtype=np.float32)
# N_tot = np.concatenate((N_tot, np.zeros((edges + edges_2,1), dtype=np.float32)),  axis=1 )
E = np.concatenate((E, np.asarray(e2)), axis=0)
N_tot = np.eye(edges * 2,  dtype=np.float32)
N_tot = np.concatenate((N_tot, np.zeros((edges * 2,1), dtype=np.float32)),  axis=1 )
# Create Input to GNN
inp, arcnode, graphnode = gnn.gnn_utils.from_EN_to_GNN(E, N_tot)
# inp, arcnode, graphnode = gnn.gnn_utils.from_EN_to_GNN(E, N)
labels = labels[..., None]

AttributeError: ignored

## ###############################################################################################



In [12]:
# set input and output dim, the maximum number of iterations, the number of epochs and the optimizer
threshold = 0.01
learning_rate = 0.01
state_dim = 3
tf.reset_default_graph()
input_dim = inp.shape[1]
output_dim = labels.shape[1]
print(input_dim)
print(output_dim)
max_it = row * column
num_epoch = 1000
optimizer = tf.train.AdamOptimizer

# initialize state and output network
net = Net(input_dim, state_dim, output_dim)
# initialize GNN
param = "st_d" + str(state_dim) + "_th" + str(threshold) + "_lr" + str(learning_rate)

tensorboard = False

g = GNN.GNN(net, input_dim, output_dim, state_dim,  max_it, optimizer, learning_rate, threshold, graph_based=False, param=param, config=config,
            tensorboard=tensorboard)

# train the model
count = 0

######

for j in range(0, num_epoch):
    _, it = g.Train(inputs=inp, ArcNode=arcnode, target=labels, step=count)

    if count % 30 == 0:
        print("Epoch ", count)
        print("Training: ", g.Validate(inp, arcnode, labels, count))

        # end = time.time()
        # print("Epoch {} at time {}".format(j, end-start))
        # start = time.time()

    count = count + 1

# evaluate on the test set
print("\nEvaluate: \n")

print("Training: ", g.Predict(inp, arcnode))
print("Expected ", labels)

#g = GNN.GNN(net, input_dim, output_dim, state_dim,  max_it, optimizer, learning_rate, threshold, graph_based=False, param=param, config=config, tensorboard=tensorboard)
#number of nodes
edges = row * column
e = grid(row, column, 0)
# undirected graph, adding other direction
e.extend([[i, j, num] for j, i, num in e])
source_id = 1
result = hop_count(source_id, e).items()
labels1 = np.array(list(map(lambda x : x[1], result)))

#reorder
e = sorted(e)
E = np.asarray(e)

# creating node features - simply one-hot values
N = np.zeros((edges, edges), dtype=np.float32)
N[source_id, source_id] = 1
# adding column thta represent the id of the graph to which the node belongs
N = np.concatenate((N, np.zeros((edges,1), dtype=np.float32)),  axis=1)

inp1, arcnode1, graphnode1 = gnn.gnn_utils.from_EN_to_GNN(E, N)

print("Training: ", g.Predict(inp1, arcnode1))
print("Expected ", labels)

38
1


/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch  0
Training:  (3.0179746, 1.0, 9)
Epoch  30
Training:  (0.48376513, 1.0, 9)
Epoch  60
Training:  (0.09562203, 1.0, 9)
Epoch  90
Training:  (0.03201969, 1.0, 9)
Epoch  120
Training:  (0.010088236, 1.0, 9)
Epoch  150
Training:  (0.0055572763, 1.0, 9)
Epoch  180
Training:  (0.0045848927, 1.0, 9)
Epoch  210
Training:  (0.0037399505, 1.0, 9)
Epoch  240
Training:  (0.0030462483, 1.0, 9)
Epoch  270
Training:  (0.0023724406, 1.0, 9)
Epoch  300
Training:  (0.0018404248, 1.0, 9)
Epoch  330
Training:  (0.0014057045, 1.0, 9)
Epoch  360
Training:  (0.0010571579, 1.0, 9)
Epoch  390
Training:  (0.00078296964, 1.0, 9)
Epoch  420
Training:  (0.00057119917, 1.0, 9)
Epoch  450
Training:  (0.00041051532, 1.0, 9)
Epoch  480
Training:  (0.0002906801, 1.0, 9)
Epoch  510
Training:  (0.00020280428, 1.0, 9)
Epoch  540
Training:  (0.00013942338, 1.0, 9)
Epoch  570
Training:  (9.444968e-05, 1.0, 9)
Epoch  600
Training:  (6.304819e-05, 1.0, 9)
Epoch  630
Training:  (4.1470958e-05, 1.0, 9)
Epoch  660
Training

ValueError: ignored